<a href="https://colab.research.google.com/github/pkRaksha/Aisle_Assignment/blob/main/Python_Aisle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing librarires

import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

#Creating a class Receipt

class Receipt:


	# initialization or constructor method of class Receipt
  def __init__(self,items_list):
    self.df = pd.DataFrame(columns=['Quantity','is_imported','Product_Description','Price'])

    for i in items_list:
      self.item=i
      self.Quantity=[]
      self.Price=[]
      self.Description=[]
      self.is_imported=0
      self.words=''
      self.creatingDataframe()
    self.Result()
  #Function to handle float values for comparison purpose
  def isfloat(self,num):
    try:
        float(num)
        return True
    except ValueError:
        return False
# Intiliazing values for the dataframe
  def creatingDataframe(self):
    if self.item!='':
      self.item=self.item.split(' ')
      for i in self.item:
        if i.isdigit():
          self.Quantity.append(i)
        elif self.isfloat(i):
          self.Price.append(i)
        elif isinstance(i, str):
          if i.lower() =='imported':
             self.is_imported=1;
          self.words = self.words+' '+i
    self.LoadingDataframe()

	# Loading all the initialized values into the dataframe appending each row
  def LoadingDataframe(self):
    self.Description=re.split(r'\bat\b', self.words)
    self.Description = [x for x in self.Description if x != '' if x != ' ']
    self.df = self.df.append({'Quantity':self.Quantity,'is_imported':self.is_imported,'Product_Description':self.Description,'Price':self.Price}, ignore_index=True)
    self.Cleaning()
# Performing feature engineering
  def Cleaning(self):
#Calculating the import duty for each product
    def Calc_import_duty(is_imported,price):
      duty=[]
      for p in price:
        if is_imported:
          result=float(p)*0.05
          duty.append(round(result,2))
        else:
          duty.append(0.0)
      return duty
    self.df['import_duty']=self.df.apply(lambda x: Calc_import_duty(x.is_imported,x.Price) , axis=1)


# calculating the sale_tax for each product
    def Calc_sales_tax(product,price):
      tax=[]
      for i in range(len(product)):
         if 'book' in product[i] or 'bar' in product[i] or 'pills' in product[i]  or'chocolates' in product[i] :
             tax.append(0)
         else:
          result=float(price[i])*0.1
          tax.append(round(result,2))

      return tax
    self.df['sales_tax']=self.df.apply(lambda x: Calc_sales_tax(x.Product_Description,x.Price),axis=1)

#Calculating the total amount by adding all the tax applied on each product
    def Calc_amount(price,duty,tax):
      total=0
      result=[]
      for i in range(len(price)):
        total=float(price[i])+float(duty[i])+float(tax[i])
        result.append(round(total,2))
      return result
    self.df['total_amount']=self.df.apply(lambda x: Calc_amount(x.Price,x.import_duty,x.sales_tax),axis=1)

 #Printing the desired result by iterating each row in the dataframe
  def Result(self):
    self.tax_sum=0
    self.overall_sum=0
    for index, row in self.df.iterrows():
      for i in range(len(row['Quantity'])):
          print(f"{row['Quantity'][i]} {row['Product_Description'][i]} :{row['total_amount'][i]}")
          self.tax_sum=self.tax_sum+row['sales_tax'][i]+row['import_duty'][i]
          self.overall_sum=self.overall_sum+row['total_amount'][i]
    print(f"Sales Taxes: {round(self.tax_sum,2)} Total: {round(self.overall_sum,2)}")


''' Getting input from user , Type q to once the desired list of products are entered '''

if __name__ == "__main__":
  product_list=[]
  while(1):
    item=input('Enter the Product detail')
    if item!='q':
      product_list.append(item)
    elif item=='q':
      break
  bill=Receipt(product_list)












Enter the Product detail1 imported bottle of perfume at 27.99 1 bottle of perfume at 18.99
Enter the Product detail1 packet of headache pills at 9.75
Enter the Product detail1 box of imported chocolates at 11.25
Enter the Product detailq
1  imported bottle of perfume  :32.19
1  bottle of perfume  :21.84
1  packet of headache pills  :9.75
1  box of imported chocolates  :11.81
Sales Taxes: 7.61 Total: 75.59
